In [3]:
import pandas as pd
from imblearn.combine import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from scipy.stats import randint as sp_randint
import numpy as np
from sklearn.grid_search import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# 언더샘플링

* TomekLinks

In [8]:
%%time


n_estimators = list(np.arange(50,151,10))
learning_rate = list(np.arange(0.01,0.13,0.01))
max_depth = [3,4,5,6,7,8,9]


def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : TomekLinks')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= {'n_estimators' : n_estimators ,
            'learning_rate' : learning_rate ,
            'max_depth' : max_depth
           }
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = TomekLinks(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


Sampling name : TomekLinks
XGBoost :  {'learning_rate': 0.099999999999999992, 'max_depth': 7, 'n_estimators': 110}
train score: -0.5293616865841884
test score: -0.8433441601249745
log loss: 0.8433441601249745

Wall time: 45min 9s


* OneSidedSelection

In [9]:
%%time
n_estimators = list(np.arange(50,151,10))
learning_rate = list(np.arange(0.01,0.13,0.01))
max_depth = [3,4,5,6,7,8,9]


def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : OneSidedSelection')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= {'n_estimators' : n_estimators ,
            'learning_rate' : learning_rate ,
            'max_depth' : max_depth
           }
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = OneSidedSelection(random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)


C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
C:\ProgramData\Anaconda3\lib\site-packages\imblearn\under_sampling\prototype_selection\one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)


Sampling name : OneSidedSelection
XGBoost :  {'learning_rate': 0.069999999999999993, 'max_depth': 7, 'n_estimators': 150}
train score: -0.5324246155834526
test score: -0.8447549785319242
log loss: 0.8447549785319242

Wall time: 44min 41s


# 오버샘플링

* ADASYN

In [7]:
%%time
n_estimators = list(np.arange(350,451,10))
learning_rate = list(np.arange(0.01,0.1,0.01))
max_depth = [4]
                

def model_cv(X_train, X_test, y_train, y_test, model_xgb, name):
    model_xgb.fit(X_train, y_train)
    print('Sampling name : ADASYN')
    print(name,': ',model_xgb.best_params_)
    pred_y = model_xgb.predict_proba(X_test)
    print('train score: {}'.format(model_xgb.score(X_train, y_train)))
    print('test score: {}'.format(model_xgb.score(X_test, y_test)))
    print('log loss: {}'.format(log_loss(y_test, pred_y)))
    print()

def xgbc(X_train, X_test, y_train, y_test):
    param= {'n_estimators' : n_estimators,
            'learning_rate' : learning_rate,
            'max_depth' : max_depth
           }
    model_xgb = GridSearchCV(XGBClassifier(), param, scoring="neg_log_loss",n_jobs = 4)
    name = 'XGBoost'
    return model_cv(X_train, X_test, y_train, y_test, model_xgb, name)


df_imb= pd.read_csv('df_pre.csv')
del df_imb['Unnamed: 0']


columns=['SexuponOutcome','Color_pre','DayuponOutcome','Breed_pre','Name','DateTime' ,'AnimalType','BreedMix']

df_imb_x = df_imb[columns]
df_imb_y = df_imb.iloc[:, [3]]
X_train, X_test, y_train, y_test = train_test_split(df_imb_x, df_imb_y, test_size = 0.3, random_state=0, shuffle=True)

X, y = ADASYN(ratio = 'minority',random_state=0).fit_sample(X_train, y_train)
final_X = pd.DataFrame(X, columns=columns)

xgbc(final_X, X_test, y, y_test)

Sampling name : ADASYN
XGBoost :  {'learning_rate': 0.080000000000000002, 'max_depth': 4, 'n_estimators': 450}
train score: -0.5217376174557616
test score: -0.8217058350824629
log loss: 0.8217058350824629

Wall time: 28min 12s
